# Part 3: Automated Testing

**Class 3 — Non-Functional Testing & AI Security**

---

## Objective

1. Write **pytest scripts** for three test categories:
   - API response tests (correctness & error handling)
   - Accuracy threshold tests (model quality gate)
   - Latency threshold tests (performance SLA)
2. Understand **Continuous Integration (CI)** and how automated tests gate deployment
3. See how an **automated ML testing pipeline** works end-to-end

---

## Key Concepts

| Concept | Definition |
|---------|------------|
| **pytest** | Python test framework — auto-discovers `test_*.py` files and `test_*` functions |
| **Fixture** | Reusable test setup/teardown declared with `@pytest.fixture` |
| **Assertion** | `assert <condition>` — test fails if condition is False |
| **CI/CD** | Continuous Integration/Delivery — run tests on every code push |
| **Quality Gate** | Minimum passing criteria before code can merge to main |
| **SLA** | Service Level Agreement — e.g., p99 latency < 200ms |

---

## The Testing Pyramid for ML Systems

```
        ┌──────────────────┐
        │  End-to-End      │  ← Slow, few: full prediction workflow
        ├──────────────────┤
        │  Integration     │  ← Medium: multi-model, canary, metrics
        ├──────────────────┤
        │  API Response    │  ← Fast, many: endpoint correctness
        ├──────────────────┤
        │  Unit: Accuracy  │  ← Model quality gates
        ├──────────────────┤
        │  Unit: Latency   │  ← Performance SLA gates
        └──────────────────┘
```

Class 3 focuses on the **API Response**, **Accuracy**, and **Latency** layers.

In [1]:
# ── Section 1: Setup & Imports ──────────────────────────────────────────────
import os
import io
import sys
import time
import numpy as np
import pandas as pd
import pytest
from pathlib import Path
from PIL import Image
from datetime import datetime, timedelta
from typing import Optional, Dict
from collections import defaultdict

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from fastapi import FastAPI, File, UploadFile, HTTPException, Depends, Request
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from fastapi.testclient import TestClient
from pydantic import BaseModel
from jose import JWTError, jwt
import bcrypt as _bcrypt

# Constants
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CLASS_NAMES = ['animal', 'name_board', 'other_vehicle', 'pedestrian',
               'pothole', 'road_sign', 'speed_breaker']
NUM_CLASSES = len(CLASS_NAMES)
DATASET_PATH = r'C:\Users\Lucifer\python_workspace\BITS\AI_Quality_Engineering\dataset'
TEST_PATH = os.path.join(DATASET_PATH, 'test')

# Test thresholds
# NOTE: These use values compatible with RANDOM weights (no training).
# In production with a trained model, set:
#   ACCURACY_THRESHOLD = 70.0  (trained ResNet-18 achieves ~83%)
#   LATENCY_P99_SLA_MS = 100   (tighter SLA for production)
ACCURACY_THRESHOLD  = 10.0   # Random weights → ~14% (chance level)
LATENCY_P99_SLA_MS  = 500    # ms — conservative for CPU demo
SINGLE_LATENCY_MS   = 200    # ms — per-request SLA

print(f'Device: {DEVICE}')
print(f'Test thresholds: accuracy>{ACCURACY_THRESHOLD}%, latency p99<{LATENCY_P99_SLA_MS}ms')
print('All imports successful!')

Device: cpu
Test thresholds: accuracy>10.0%, latency p99<500ms
All imports successful!


In [2]:
# ── Section 2: Model & API Setup ─────────────────────────────────────────────

# Model definition (same as class2)
class ADASModel(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.resnet = resnet18(weights=None)
        self.resnet.fc = nn.Linear(512, num_classes)
    def forward(self, x): return self.resnet(x)

transform = transforms.Compose([
    transforms.Resize((128, 128)), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

model = ADASModel(NUM_CLASSES).to(DEVICE)
model.eval()

# --- JWT setup (from Part 2) ---
SECRET_KEY = 'adas-demo-secret-key-change-in-production'
ALGORITHM  = 'HS256'
oauth2_scheme = OAuth2PasswordBearer(tokenUrl='token')
FAKE_USERS_DB = {
    'testuser': {'username': 'testuser',
                 'hashed_password': _bcrypt.hashpw(b'testpassword', _bcrypt.gensalt()),
                 'disabled': False}
}

def create_access_token(data, expires_delta=None):
    to_encode = data.copy()
    expire = datetime.utcnow() + (expires_delta or timedelta(minutes=15))
    to_encode.update({'exp': expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

async def get_current_user(token: str = Depends(oauth2_scheme)):
    try:
        payload  = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username = payload.get('sub')
        if username is None: raise HTTPException(status_code=401, detail='Invalid token')
    except JWTError:
        raise HTTPException(status_code=401, detail='Invalid token',
                            headers={'WWW-Authenticate': 'Bearer'})
    user = FAKE_USERS_DB.get(username)
    if not user: raise HTTPException(status_code=401, detail='User not found')
    return user

# Pydantic schemas
class Token(BaseModel):
    access_token: str; token_type: str

class PredictionResponse(BaseModel):
    prediction: str; confidence: float
    class_probabilities: Dict[str, float]
    model_version: str; latency_ms: float

# --- FastAPI app ---
app = FastAPI(title='ADAS Test API', version='3.0.0')

@app.post('/token', response_model=Token)
async def login(form_data: OAuth2PasswordRequestForm = Depends()):
    user = FAKE_USERS_DB.get(form_data.username)
    if not user or not _bcrypt.checkpw(form_data.password.encode('utf-8'), user['hashed_password']):
        raise HTTPException(status_code=401, detail='Invalid credentials')
    token = create_access_token({'sub': user['username']}, timedelta(minutes=30))
    return {'access_token': token, 'token_type': 'bearer'}

@app.get('/health')
async def health(): return {'status': 'healthy', 'version': '3.0.0', 'device': str(DEVICE)}

@app.post('/predict', response_model=PredictionResponse)
async def predict(file: UploadFile = File(...), current_user: dict = Depends(get_current_user)):
    start = time.time()
    allowed = {'.jpg', '.jpeg', '.png', '.gif', '.bmp'}
    ext = Path(file.filename).suffix.lower() if file.filename else ''
    if ext not in allowed:
        raise HTTPException(status_code=400, detail=f'Invalid file type: {ext}')
    contents = await file.read()
    if not contents: raise HTTPException(status_code=400, detail='Empty file')
    try: image = Image.open(io.BytesIO(contents)).convert('RGB')
    except Exception as e: raise HTTPException(status_code=400, detail=f'Invalid image: {e}')
    if image.size[0] < 32 or image.size[1] < 32:
        raise HTTPException(status_code=400, detail='Image too small (min 32x32)')
    tensor = transform(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        out   = model(tensor)
        probs = torch.softmax(out, dim=1)
        conf, idx = torch.max(probs, 1)
    lat = (time.time() - start) * 1000
    return PredictionResponse(
        prediction=CLASS_NAMES[idx.item()], confidence=round(float(conf.item()),4),
        class_probabilities={CLASS_NAMES[i]: float(probs[0,i]) for i in range(NUM_CLASSES)},
        model_version='v3.0', latency_ms=round(lat,2))

client = TestClient(app)

# Pre-acquire token for authenticated tests
token_resp = client.post('/token', data={'username': 'testuser', 'password': 'testpassword'})
AUTH_TOKEN  = token_resp.json()['access_token']
AUTH_HEADERS = {'Authorization': f'Bearer {AUTH_TOKEN}'}

print('API and TestClient ready.')
print(f'Auth token acquired: {AUTH_TOKEN[:30]}...')

c:\Users\Lucifer\anaconda3\envs\venv_ai_quality\lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_version" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


API and TestClient ready.
Auth token acquired: eyJhbGciOiJIUzI1NiIsInR5cCI6Ik...


## Section 3: Test Helpers

Helper functions used across all test classes.

In [3]:
# ── Section 3: Test Helpers ──────────────────────────────────────────────────

def create_test_image(size=(100, 100), color='red', fmt='PNG'):
    """Create an in-memory test image. Returns (BytesIO, filename)."""
    img = Image.new('RGB', size, color=color)
    buf = io.BytesIO()
    img.save(buf, format=fmt)
    buf.seek(0)
    return buf, f'test.{fmt.lower()}'


def post_predict(image_buf, filename='test.png'):
    """POST to /predict with authentication."""
    return client.post(
        '/predict',
        headers=AUTH_HEADERS,
        files={'file': (filename, image_buf, 'image/png')},
    )


def evaluate_model_accuracy(eval_model, loader, device):
    """Evaluate accuracy on a DataLoader. Returns percentage."""
    eval_model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            _, preds = torch.max(eval_model(images), 1)
            correct += (preds.cpu() == labels).sum().item()
            total   += labels.size(0)
    return round(100 * correct / total, 2) if total > 0 else 0.0


# Test helper functions
buf, fname = create_test_image()
resp = post_predict(buf, fname)
print(f'Helper sanity check: {resp.status_code} → {resp.json().get("prediction", "error")}')

Helper sanity check: 200 → pothole


## Section 4: Category 1 — API Response Tests

These tests verify that the API:
- Returns correct HTTP status codes
- Includes required fields in responses
- Handles errors gracefully
- Enforces security (401 without token)

**pytest convention**: Test methods start with `test_`, organized in classes starting with `Test`.

In [4]:
# ── Section 4: API Response Tests ────────────────────────────────────────────

class TestAPIResponse:
    """
    Tests for API endpoint correctness, error handling, and security.
    Category 1 of 3 in our automated test suite.
    """

    # -- Health endpoint --

    def test_health_endpoint_returns_200(self):
        resp = client.get('/health')
        assert resp.status_code == 200, f'Expected 200, got {resp.status_code}'

    def test_health_response_has_status_field(self):
        resp = client.get('/health')
        assert 'status' in resp.json(), 'Missing status field'
        assert resp.json()['status'] == 'healthy'

    def test_health_response_has_version_field(self):
        resp = client.get('/health')
        assert 'version' in resp.json(), 'Missing version field'

    # -- Prediction endpoint - success cases --

    def test_predict_returns_200_for_valid_image(self):
        buf, fname = create_test_image()
        resp = post_predict(buf, fname)
        assert resp.status_code == 200, f'Expected 200, got {resp.status_code}: {resp.text}'

    def test_predict_response_includes_prediction_field(self):
        buf, fname = create_test_image()
        data = post_predict(buf, fname).json()
        assert 'prediction' in data, 'Response missing prediction field'
        assert data['prediction'] in CLASS_NAMES, \
            f'Unknown prediction: {data["prediction"]}'

    def test_predict_response_includes_confidence(self):
        buf, fname = create_test_image()
        data = post_predict(buf, fname).json()
        assert 'confidence' in data
        assert 0.0 <= data['confidence'] <= 1.0, \
            f'Confidence {data["confidence"]} out of range [0, 1]'

    def test_predict_class_probabilities_sum_to_one(self):
        buf, fname = create_test_image()
        data = post_predict(buf, fname).json()
        assert 'class_probabilities' in data
        total = sum(data['class_probabilities'].values())
        assert abs(total - 1.0) < 0.01, \
            f'Probabilities sum to {total:.4f}, expected 1.0'

    def test_predict_returns_all_class_names(self):
        buf, fname = create_test_image()
        data = post_predict(buf, fname).json()
        for cls in CLASS_NAMES:
            assert cls in data['class_probabilities'], \
                f'Class {cls} missing from probabilities'

    def test_predict_includes_latency_ms(self):
        buf, fname = create_test_image()
        data = post_predict(buf, fname).json()
        assert 'latency_ms' in data
        assert data['latency_ms'] > 0, 'Latency must be positive'

    # -- Error handling --

    def test_invalid_file_type_returns_400(self):
        txt_file = io.BytesIO(b'this is not an image')
        resp = client.post(
            '/predict', headers=AUTH_HEADERS,
            files={'file': ('document.txt', txt_file, 'text/plain')},
        )
        assert resp.status_code == 400, f'Expected 400, got {resp.status_code}'

    def test_corrupt_image_returns_400(self):
        corrupt = io.BytesIO(b'this-is-not-a-valid-png-header')
        resp = client.post(
            '/predict', headers=AUTH_HEADERS,
            files={'file': ('corrupt.png', corrupt, 'image/png')},
        )
        assert resp.status_code == 400

    def test_too_small_image_returns_400(self):
        buf, fname = create_test_image(size=(16, 16))  # Below 32x32 minimum
        resp = post_predict(buf, fname)
        assert resp.status_code == 400, f'Expected 400 for tiny image, got {resp.status_code}'

    def test_empty_file_returns_400(self):
        empty = io.BytesIO(b'')
        resp = client.post(
            '/predict', headers=AUTH_HEADERS,
            files={'file': ('empty.png', empty, 'image/png')},
        )
        assert resp.status_code == 400

    # -- Security: authentication --

    def test_unauthenticated_request_returns_401(self):
        buf, fname = create_test_image()
        resp = client.post('/predict', files={'file': (fname, buf, 'image/png')})
        assert resp.status_code == 401, \
            f'Expected 401 without token, got {resp.status_code}'

    def test_wrong_credentials_returns_401(self):
        resp = client.post('/token', data={'username': 'testuser', 'password': 'wrong'})
        assert resp.status_code == 401

    def test_valid_credentials_return_token(self):
        resp = client.post('/token', data={'username': 'testuser', 'password': 'testpassword'})
        assert resp.status_code == 200
        data = resp.json()
        assert 'access_token' in data
        assert data['token_type'] == 'bearer'


print('TestAPIResponse class defined with 16 tests.')

TestAPIResponse class defined with 16 tests.


## Section 5: Category 2 — Accuracy Threshold Tests

These tests verify that the model meets a **minimum accuracy requirement** before deployment.

This is a **quality gate** — if accuracy drops below the threshold, the CI pipeline fails and the code cannot merge.

> **Note on thresholds**: We use `ACCURACY_THRESHOLD = 10%` here because we're using random weights (chance level ≈ 14.3% for 7 classes). In production with a **trained model** (ResNet-18 achieves ~83%), set `ACCURACY_THRESHOLD = 70.0`.

In [5]:
# ── Section 5: Accuracy Threshold Tests ─────────────────────────────────────

class TestAccuracyThreshold:
    """
    Model quality gate tests.
    These tests BLOCK deployment if accuracy falls below threshold.

    Threshold values:
      - Demo (random weights): 10%  (chance level = 14.3%)
      - Production (trained):  70%  (trained ResNet-18 achieves ~83%)
    """

    # Shared test dataset (loaded once)
    _test_loader = None

    @classmethod
    def get_test_loader(cls):
        if cls._test_loader is None:
            test_ds = ImageFolder(TEST_PATH, transform=transform)
            cls._test_loader = DataLoader(
                test_ds, batch_size=32, shuffle=False, num_workers=0
            )
        return cls._test_loader

    def test_overall_accuracy_above_threshold(self):
        """
        Overall accuracy on test set must exceed ACCURACY_THRESHOLD.
        PRODUCTION: Set ACCURACY_THRESHOLD = 70.0
        """
        loader = self.get_test_loader()
        accuracy = evaluate_model_accuracy(model, loader, DEVICE)
        print(f'  Overall accuracy: {accuracy:.2f}% (threshold: {ACCURACY_THRESHOLD}%)')
        assert accuracy >= ACCURACY_THRESHOLD, \
            f'Accuracy {accuracy:.2f}% below threshold {ACCURACY_THRESHOLD}%'

    def test_per_class_accuracy_not_zero(self):
        """
        Each class must be predicted at least once on the test set.
        Catches cases where the model always predicts a single class.
        """
        loader = self.get_test_loader()
        predicted_classes = set()
        model.eval()
        with torch.no_grad():
            for images, _ in loader:
                _, preds = torch.max(model(images.to(DEVICE)), 1)
                predicted_classes.update(preds.cpu().numpy().tolist())
        unique_preds = len(predicted_classes)
        print(f'  Unique classes predicted: {unique_preds}/{NUM_CLASSES}')
        assert unique_preds > 1, \
            f'Model only predicts {unique_preds} class(es) — possible degenerate model'

    def test_confidence_values_in_valid_range(self):
        """
        All class probability values must be in [0, 1] and sum to ~1.0.
        Validates that softmax is applied correctly.
        """
        for color in ['red', 'green', 'blue', 'white']:
            buf, fname = create_test_image(color=color)
            data = post_predict(buf, fname).json()
            assert 0.0 <= data['confidence'] <= 1.0, \
                f'Confidence {data["confidence"]} out of range'
            total = sum(data['class_probabilities'].values())
            assert abs(total - 1.0) < 0.01, \
                f'Probabilities sum to {total:.4f} for {color} image'

    def test_model_prediction_deterministic(self):
        """
        Same image must always produce the same prediction (no randomness).
        Verifies model is in eval() mode (no dropout active).
        """
        predictions = []
        for _ in range(5):
            buf, fname = create_test_image(color='purple', size=(128, 128))
            data = post_predict(buf, fname).json()
            predictions.append(data['prediction'])
        assert len(set(predictions)) == 1, \
            f'Non-deterministic predictions: {predictions}'

    def test_prediction_is_valid_class(self):
        """Prediction must always be one of the 7 known class names."""
        for _ in range(3):
            buf, fname = create_test_image()
            data = post_predict(buf, fname).json()
            assert data['prediction'] in CLASS_NAMES, \
                f'Unknown prediction class: {data["prediction"]}'


print('TestAccuracyThreshold class defined with 5 tests.')

TestAccuracyThreshold class defined with 5 tests.


## Section 6: Category 3 — Latency Threshold Tests

These tests verify that the API meets **performance SLA** requirements.

**Why test latency?** An ADAS car traveling at 60 km/h moves 16 meters/second. A 500ms delay means the car has moved 8 meters without updated hazard detection.

**SLA values used here** (for CPU demo with random weights):
- Single request: < 200ms
- p99 across 50 requests: < 500ms

In production: tighten to <50ms single, <100ms p99.

In [6]:
# ── Section 6: Latency Threshold Tests ──────────────────────────────────────

class TestLatencyThreshold:
    """
    Performance SLA tests.
    Verify that inference meets latency requirements for production deployment.

    SLA values (CPU demo):
      - Single request: < 200ms
      - p99 (50 requests): < 500ms
    PRODUCTION: Tighten to 50ms single, 100ms p99 with GPU.
    """

    def test_single_image_latency_under_sla(self):
        """
        A single prediction request (wall-clock) must complete within SINGLE_LATENCY_MS.
        This includes: HTTP overhead + image parsing + inference.
        """
        buf, fname = create_test_image(size=(128, 128))
        start   = time.perf_counter()
        resp    = post_predict(buf, fname)
        elapsed = (time.perf_counter() - start) * 1000
        assert resp.status_code == 200
        print(f'  Single request latency: {elapsed:.2f}ms (SLA: <{SINGLE_LATENCY_MS}ms)')
        assert elapsed < SINGLE_LATENCY_MS, \
            f'Latency {elapsed:.2f}ms exceeds SLA of {SINGLE_LATENCY_MS}ms'

    def test_p99_latency_under_sla(self):
        """
        p99 latency across 50 requests must be under LATENCY_P99_SLA_MS.
        Tail latency matters: 1% of users experiencing > 500ms is unacceptable in ADAS.
        """
        n_requests = 50
        latencies  = []
        for _ in range(n_requests):
            buf, fname = create_test_image()
            start   = time.perf_counter()
            resp    = post_predict(buf, fname)
            elapsed = (time.perf_counter() - start) * 1000
            assert resp.status_code == 200
            latencies.append(elapsed)

        p50 = np.percentile(latencies, 50)
        p95 = np.percentile(latencies, 95)
        p99 = np.percentile(latencies, 99)
        print(f'  p50={p50:.1f}ms  p95={p95:.1f}ms  p99={p99:.1f}ms  '
              f'(SLA: p99<{LATENCY_P99_SLA_MS}ms)')
        assert p99 < LATENCY_P99_SLA_MS, \
            f'p99 latency {p99:.2f}ms exceeds SLA of {LATENCY_P99_SLA_MS}ms'

    def test_latency_reported_by_api_is_positive(self):
        """
        The latency_ms field in the API response must be > 0.
        Checks that we haven't accidentally disabled timing.
        """
        buf, fname = create_test_image()
        data = post_predict(buf, fname).json()
        assert data['latency_ms'] > 0, 'API-reported latency must be positive'

    def test_large_image_within_sla(self):
        """
        Large images (640x480) should still complete within 2× the single SLA.
        Resize transform makes input size uniform — latency should not blow up.
        """
        buf, fname = create_test_image(size=(640, 480))
        start   = time.perf_counter()
        resp    = post_predict(buf, fname)
        elapsed = (time.perf_counter() - start) * 1000
        assert resp.status_code == 200
        sla_large = SINGLE_LATENCY_MS * 2
        print(f'  640x480 image latency: {elapsed:.2f}ms (SLA: <{sla_large}ms)')
        assert elapsed < sla_large, \
            f'Large image latency {elapsed:.2f}ms exceeds 2x SLA of {sla_large}ms'

    def test_successive_requests_consistent_latency(self):
        """
        Latency must not degrade significantly over successive requests.
        Checks for memory leaks or state accumulation.
        """
        latencies = []
        for _ in range(20):
            buf, fname = create_test_image()
            start = time.perf_counter()
            post_predict(buf, fname)
            latencies.append((time.perf_counter() - start) * 1000)

        first_half_mean = np.mean(latencies[:10])
        second_half_mean = np.mean(latencies[10:])
        degradation_pct = ((second_half_mean - first_half_mean) / first_half_mean) * 100
        print(f'  Latency: first 10 mean={first_half_mean:.1f}ms, '
              f'last 10 mean={second_half_mean:.1f}ms, '
              f'degradation={degradation_pct:.1f}%')
        # Allow up to 50% degradation (warming up effects on cold start)
        assert degradation_pct < 50, \
            f'Latency degraded by {degradation_pct:.1f}% over 20 requests (possible memory leak)'


print('TestLatencyThreshold class defined with 5 tests.')

TestLatencyThreshold class defined with 5 tests.


## Section 7: Run All Tests

We run all three test categories and print a summary report.

In [7]:
# ── Section 7: Test Runner ───────────────────────────────────────────────────

def run_test_class(test_class):
    """
    Run all test_* methods in a test class.
    Returns (passed, failed, results_list).
    """
    instance = test_class()
    results  = []
    methods  = [m for m in dir(instance) if m.startswith('test_')]

    for method_name in sorted(methods):
        method = getattr(instance, method_name)
        try:
            method()
            results.append((method_name, 'PASS', None))
        except AssertionError as e:
            results.append((method_name, 'FAIL', str(e)))
        except Exception as e:
            results.append((method_name, 'ERROR', str(e)))

    passed = sum(1 for _, s, _ in results if s == 'PASS')
    failed = len(results) - passed
    return passed, failed, results


def print_test_results(class_name, passed, failed, results):
    total = passed + failed
    status = '✓ ALL PASS' if failed == 0 else f'✗ {failed} FAILED'
    print(f'\n  {class_name}: {passed}/{total} passed  [{status}]')
    for name, state, err in results:
        icon = '  ✓' if state == 'PASS' else '  ✗'
        print(f'{icon} {name}')
        if err: print(f'     → {err[:80]}')


# Run all test categories
print('=' * 65)
print('  Automated Test Suite — Class 3 Part 3')
print('=' * 65)

total_pass = total_fail = 0

for cls in [TestAPIResponse, TestAccuracyThreshold, TestLatencyThreshold]:
    p, f, r = run_test_class(cls)
    print_test_results(cls.__name__, p, f, r)
    total_pass += p
    total_fail += f

print('\n' + '=' * 65)
print(f'  TOTAL: {total_pass} passed, {total_fail} failed')
overall = 'ALL TESTS PASSED ✓' if total_fail == 0 else f'{total_fail} TESTS FAILED ✗'
print(f'  {overall}')
print('=' * 65)

  Automated Test Suite — Class 3 Part 3

  TestAPIResponse: 16/16 passed  [✓ ALL PASS]
  ✓ test_corrupt_image_returns_400
  ✓ test_empty_file_returns_400
  ✓ test_health_endpoint_returns_200
  ✓ test_health_response_has_status_field
  ✓ test_health_response_has_version_field
  ✓ test_invalid_file_type_returns_400
  ✓ test_predict_class_probabilities_sum_to_one
  ✓ test_predict_includes_latency_ms
  ✓ test_predict_response_includes_confidence
  ✓ test_predict_response_includes_prediction_field
  ✓ test_predict_returns_200_for_valid_image
  ✓ test_predict_returns_all_class_names
  ✓ test_too_small_image_returns_400
  ✓ test_unauthenticated_request_returns_401
  ✓ test_valid_credentials_return_token
  ✓ test_wrong_credentials_returns_401
  Overall accuracy: 23.93% (threshold: 10.0%)
  Unique classes predicted: 1/7

  TestAccuracyThreshold: 4/5 passed  [✗ 1 FAILED]
  ✓ test_confidence_values_in_valid_range
  ✓ test_model_prediction_deterministic
  ✓ test_overall_accuracy_above_threshold
  

## Section 8: CI/CD — Continuous Integration Concept

### What is CI/CD?

```
Developer                GitHub                  CI Server
    │                      │                          │
    ├── git push ──────────►│                          │
    │                       ├── trigger pipeline ──────►│
    │                       │                          │
    │                       │                          │ pip install
    │                       │                          │ pytest class3/
    │                       │                          │   ✓ 26 passed
    │                       │◄─── PASS → allow merge ──┤
    │◄── PR green ──────────┤                          │
```

**Key principle**: No code reaches `main` branch without passing all tests.

### GitHub Actions Workflow

This is the YAML file you'd commit to `.github/workflows/ml-tests.yml`:

In [8]:
# ── Section 8: CI/CD Pipeline Configuration ──────────────────────────────────
# This YAML would live at .github/workflows/ml-tests.yml in your repository.
# It triggers automatically on every push and pull request.

github_actions_yaml = '''
# .github/workflows/ml-tests.yml
# ML Quality Gate — runs on every push and pull request

name: ML Quality Gate

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]

jobs:
  test:
    runs-on: ubuntu-latest
    timeout-minutes: 15

    steps:
      # 1. Check out code
      - name: Checkout repository
        uses: actions/checkout@v4

      # 2. Set up Python
      - name: Set up Python 3.10
        uses: actions/setup-python@v4
        with:
          python-version: "3.10"
          cache: pip

      # 3. Install dependencies
      - name: Install dependencies
        run: pip install -r class3/requirements-class3.txt

      # 4. Run API response tests
      - name: Run API response tests
        run: pytest class3/test_production.py::TestAPIResponse -v --tb=short

      # 5. Run accuracy quality gate
      - name: Run accuracy threshold tests
        run: pytest class3/test_production.py::TestAccuracyThreshold -v --tb=short

      # 6. Run latency SLA tests
      - name: Run latency SLA tests
        run: pytest class3/test_production.py::TestLatencyThreshold -v --tb=short

      # 7. Generate HTML report (artifact)
      - name: Generate test report
        run: pytest class3/test_production.py -v --html=test-report.html --self-contained-html
        if: always()  # Run even if tests fail

      # 8. Upload report for review
      - name: Upload test report
        uses: actions/upload-artifact@v4
        if: always()
        with:
          name: test-report
          path: test-report.html
          retention-days: 30
'''

print(github_actions_yaml)

print('\n📋 Key points about this CI pipeline:')
print('  1. Runs on EVERY push to main/develop and every pull request')
print('  2. Tests are split by category for clearer failure diagnosis')
print('  3. HTML report is uploaded even when tests fail (post-mortem analysis)')
print('  4. 15-minute timeout prevents runaway jobs from blocking the pipeline')
print('  5. Python version is pinned to 3.10 to match production environment')


# .github/workflows/ml-tests.yml
# ML Quality Gate — runs on every push and pull request

name: ML Quality Gate

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]

jobs:
  test:
    runs-on: ubuntu-latest
    timeout-minutes: 15

    steps:
      # 1. Check out code
      - name: Checkout repository
        uses: actions/checkout@v4

      # 2. Set up Python
      - name: Set up Python 3.10
        uses: actions/setup-python@v4
        with:
          python-version: "3.10"
          cache: pip

      # 3. Install dependencies
      - name: Install dependencies
        run: pip install -r class3/requirements-class3.txt

      # 4. Run API response tests
      - name: Run API response tests
        run: pytest class3/test_production.py::TestAPIResponse -v --tb=short

      # 5. Run accuracy quality gate
      - name: Run accuracy threshold tests
        run: pytest class3/test_production.py::TestAccuracyThreshold -v --tb=short

      # 6. Run latency SLA te

In [9]:
# ── Automated ML Testing Pipeline Summary ────────────────────────────────────

pipeline_stages = [
    ('Code Push',         'Developer pushes model update or API change',         'Automatic'),
    ('Dependency Install','pip install -r requirements-class3.txt',               'Automatic'),
    ('API Response Tests','16 tests: endpoints, error handling, auth',            'Automatic'),
    ('Accuracy Gate',     '5 tests: accuracy ≥ threshold, determinism',           'Automatic'),
    ('Latency Gate',      '5 tests: p99 < SLA, degradation check',               'Automatic'),
    ('Report Generation', 'HTML report uploaded as artifact',                     'Automatic'),
    ('Merge Decision',    'PR blocked if any test fails; auto-merge if all pass', 'Automatic'),
    ('Model Deploy',      'Trigger Docker build + deploy on merge to main',       'Automatic'),
]

df_pipeline = pd.DataFrame(pipeline_stages, columns=['Stage', 'Action', 'Trigger'])
df_pipeline.index += 1

print('\nAutomated ML Testing Pipeline')
print('=' * 70)
print(df_pipeline.to_string())
print('\nTotal automated checks: 26 tests across 3 categories')
print('Estimated pipeline runtime: 3-5 minutes on a standard CI runner')


Automated ML Testing Pipeline
                Stage                                                Action    Trigger
1           Code Push           Developer pushes model update or API change  Automatic
2  Dependency Install                pip install -r requirements-class3.txt  Automatic
3  API Response Tests             16 tests: endpoints, error handling, auth  Automatic
4       Accuracy Gate            5 tests: accuracy ≥ threshold, determinism  Automatic
5        Latency Gate                 5 tests: p99 < SLA, degradation check  Automatic
6   Report Generation                      HTML report uploaded as artifact  Automatic
7      Merge Decision  PR blocked if any test fails; auto-merge if all pass  Automatic
8        Model Deploy        Trigger Docker build + deploy on merge to main  Automatic

Total automated checks: 26 tests across 3 categories
Estimated pipeline runtime: 3-5 minutes on a standard CI runner


## Summary & Key Takeaways

### What We Built

| Test Category | Tests | Purpose |
|---------------|-------|--------|
| **API Response** | 16 | Correctness, error handling, auth (401/400) |
| **Accuracy Threshold** | 5 | Model quality gate — block deployment if accuracy drops |
| **Latency Threshold** | 5 | SLA gate — block deployment if latency exceeds limit |
| **Total** | **26** | Comprehensive non-functional test coverage |

### pytest Best Practices

1. **Test one thing per function** — `test_health_returns_200` not `test_health`
2. **Descriptive names** — name says what it tests AND what the expected behavior is
3. **Use fixtures** — `@pytest.fixture` for reusable setup (test images, tokens, DB state)
4. **Arrange-Act-Assert (AAA)** — setup → call → verify
5. **Test edge cases** — empty files, corrupt images, missing fields, boundary values

### CI/CD for ML Systems

A mature ML CI pipeline includes:
- **Functional tests**: Does the API work correctly?
- **Non-functional tests**: Is it fast enough? Accurate enough? Secure?
- **Data validation**: Is the training data within expected distribution?
- **Model regression**: Did the new model do WORSE than the current production model?
- **Security scan**: Does the code have known vulnerabilities? (bandit, safety)

### Next Step: Lab 3

Now it's your turn. Lab 3 combines everything from Parts 1–3 into a **Production-Grade Testing Checklist**.